In [ ]:
import math
import os
import shutil
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import pydicom
import cv2
import json
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from skimage import exposure
from ast import literal_eval
from matplotlib.patches import Rectangle

In [ ]:
random_stat = 123
np.random.seed(random_stat)

In [ ]:
!git clone https://github.com/pjreddie/darknet.git
!apt-get install -y python-opencv

%cd darknet

!make

In [ ]:
print(os.getcwd())

In [ ]:
DATA_DIR = "../../input/siim-covid19-detection"

train_dcm_dir = os.path.join(DATA_DIR, "train")
test_dcm_dir = os.path.join(DATA_DIR, "test")

IMAGE_DIR = '../../input/covid19jpg/CovidJPG'

label_dir = os.path.join(os.getcwd(), "labels")  # .txt
metadata_dir = os.path.join(os.getcwd(), "metadata") # .txt

# YOLOv3 config file directory
cfg_dir = os.path.join(os.getcwd(), "cfg")
# YOLOv3 training checkpoints will be saved here
backup_dir = os.path.join(os.getcwd(), "backup")

for directory in [label_dir, metadata_dir, cfg_dir, backup_dir]:
    if os.path.isdir(directory):
        continue
    os.mkdir(directory)

### 2.2. Generate images and labels for training YOLOv3
* YOLOv3 needs .txt file for each image, which contains ground truth object in the image that looks like:
```
<object-class_1> <x_1> <y_1> <width_1> <height_1>
<object-class_2> <x_2> <y_2> <width_2> <height_2>
```
* <object-class\>: Since RSNA task is binary classification basically, <object-class\> is 0.
* <x\>, <y\>: Those are float values of bbox center coordinate, divided by image width and height respectively.
* <w\>, <h\>: Those are width and height of bbox, divided by image width and height respectively.

* So it is different from the format of label data provided by kaggle. We should change it.

In [ ]:
df_image = pd.read_csv(os.path.join(DATA_DIR, 'train_image_level.csv'))
df_study = pd.read_csv(os.path.join(DATA_DIR, 'train_study_level.csv'))
df_study['id'] = df_study['id'].str.replace('_study',"")
df_study.rename({'id': 'StudyInstanceUID'},axis=1, inplace=True)
df_train = df_image.merge(df_study, on='StudyInstanceUID')
df_train.loc[df_train['Negative for Pneumonia']==1, 'study_label'] = 'negative'
df_train.loc[df_train['Typical Appearance']==1, 'study_label'] = 'typical'
df_train.loc[df_train['Indeterminate Appearance']==1, 'study_label'] = 'indeterminate'
df_train.loc[df_train['Atypical Appearance']==1, 'study_label'] = 'atypical'
df_train.drop(['Negative for Pneumonia','Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance'], axis=1, inplace=True)
df_train['id'] = df_train['id'].str.replace('_image', '.jpg')
df_train['image_label'] = df_train['label'].str.split().apply(lambda x : x[0])
df_size = pd.read_csv('../../input/covid19jpg/CovidJPG/size.csv')
df_train = df_train.merge(df_size, on='id')
df_train.head(3)


In [ ]:
train_dir = os.path.join(IMAGE_DIR,'train')
test_dir = os.path.join(IMAGE_DIR, 'test')

def preprocess_image(img):
    equ_img = exposure.equalize_adapthist(img/255, clip_limit=0.05, kernel_size=24)
    return equ_img

df_opa = df_train[df_train['image_label']=='opacity'].reset_index()
fig, axs = plt.subplots(5, 2, figsize=(10,20))
fig.subplots_adjust(hspace=.2, wspace=.2)
n=5
for i in range(n):
    img = cv2.imread(os.path.join(train_dir, df_opa['id'][i]))
    img_proc = preprocess_image(img)
    axs[i, 0].imshow(img)
    axs[i, 1].imshow(img_proc)
    axs[i, 0].axis('off')
    axs[i, 1].axis('off')
    boxes = literal_eval(df_opa['boxes'][i])
    for box in boxes:
        axs[i, 0].add_patch(Rectangle((box['x']*(512/df_opa['dim1'][i]), box['y']*(512/df_opa['dim0'][i])), box['width']*(512/df_opa['dim1'][i]), box['height']*(512/df_opa['dim0'][i]), fill=0, color='y', linewidth=3))
        axs[i, 0].set_title(df_opa['study_label'][i])
        axs[i, 1].add_patch(Rectangle((box['x']*(512/df_opa['dim1'][i]), box['y']*(512/df_opa['dim0'][i])), box['width']*(512/df_opa['dim1'][i]), box['height']*(512/df_opa['dim0'][i]), fill=0, color='r', linewidth=3))
        axs[i, 1].set_title('After CLAHE')
    
plt.show()

In [ ]:
def isNaN(string):
    return string != string

def save_label(df, img_dir):
    img_size = 512
    
    for index, row in df.iterrows():
        if isNaN(row['boxes']):
            continue
        im_path = os.path.join(img_dir, row['id'])
        label_path = os.path.join(label_dir, "{}.txt".format(row['id'].split('.')[0]))
        if row['image_label'] == "opacity":
            image_label = 1
        else:
            image_label = 0
        f = open(label_path, "a")
        boxes=row['boxes']
        boxes = boxes.strip('[{')
        boxes = boxes.strip('}]')
        box_split = boxes.split('},')
        my_boxes=[]
        for box in box_split:
            box = box.strip(' {')
            box = box.strip('}')
            box_elem = box.split(',')
            my_box=[]
            for elem in box_elem:
                my_box.append(elem.split(': ')[1])
                
            top_left_x = float(my_box[0]) * 512/row['dim1']
            top_left_y = float(my_box[1]) * 512/row['dim0']
            w = float(my_box[2]) * 512/row['dim1']
            h = float(my_box[3]) * 512/row['dim0']

            rx = top_left_x/img_size
            ry = top_left_y/img_size
            rw = w/img_size
            rh = h/img_size
            rcx = rx+rw/2
            rcy = ry+rh/2

            line = "{} {} {} {} {}\n".format(image_label, rcx, rcy, rw, rh)
            f.write(line)            
        f.close()

In [ ]:
save_label(df_train, train_dir)

In [ ]:
!rm -rf ../../input/covid19jpg/CovidJPG

In [ ]:
ex_patient_id = '00326161e51e'
ex_img_path = os.path.join(train_dir, "{}.jpg".format(ex_patient_id))
ex_label_path = os.path.join(label_dir, "{}.txt".format(ex_patient_id))

plt.imshow(cv2.imread(ex_img_path))

img_size = 512
with open(ex_label_path, "r") as f:
    for line in f:
        print(line)
        class_id, rcx, rcy, rw, rh = list(map(float, line.strip().split()))
        x = (rcx-rw/2)*img_size
        y = (rcy-rh/2)*img_size
        w = rw*img_size
        h = rh*img_size
        plt.plot([x, x, x+w, x+w, x], [y, y+h, y+h, y, y])

In [ ]:
def write_train_list(metadata_dir, img_dir, name, series):
    list_fp = os.path.join(metadata_dir, name)
    with open(list_fp, "w") as f:
        for index, patient in series.iterrows():
            line = "{}\n".format(os.path.join(img_dir, "{}.jpg".format(patient['id'].split('.')[0])))
            f.write(line)

In [ ]:
series = pd.read_csv(os.path.join(IMAGE_DIR, 'size.csv'))
tr_series = series.loc[series['split'] == 'train']
val_series = series.loc[series['split'] == 'test']

# train image path list
write_train_list(metadata_dir, train_dir, "tr_list.txt", tr_series)
# validation image path list
write_train_list(metadata_dir, test_dir, "val_list.txt", val_series)

In [ ]:
ex_patient_id = 'a29c5a68b07b'
ex_img_path = os.path.join(test_dir, "{}.jpg".format(ex_patient_id))

plt.imshow(cv2.imread(ex_img_path))

In [ ]:
data_extention_file_path = os.path.join(cfg_dir, 'rsna.data')
with open(data_extention_file_path, 'w') as f:
    contents = """classes= 1
train  = {}
valid  = {}
names  = {}
backup = {}
    """.format(os.path.join(metadata_dir, "tr_list.txt"),
               os.path.join(metadata_dir, "val_list.txt"),
               os.path.join(cfg_dir, 'rsna.names'),
               backup_dir)
    f.write(contents)

In [ ]:
!cat cfg/rsna.data

In [ ]:
!wget -q https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
!wget --no-check-certificate -q "https://docs.google.com/uc?export=download&id=18ptTK4Vbeokqpux8Onr0OmwUP9ipmcYO" -O cfg/rsna_yolov3.cfg_train

In [ ]:
!ls ../../input/covid19jpg/CovidJPG

## 4. Training YOLOv3

### 4.0. Command for training with Pre-trained CNN Weights (darknet53.conv.74)
* I didn't run following command on kaggle kernel becuase of the long output.
* If you crash with  'CUDA Error: out of memory', Solve it by Editing 'batch' and 'subdivisions' in 'cfg/rsna_yolov3.cfg_train'
* If 'batch' and 'subdivisions' are 64 and 64 respectively, for every iteration only one image will be loaded on GPU memory. So it will use less GPU memory.

In [ ]:
!./darknet detector train cfg/rsna.data cfg/rsna_yolov3.cfg_train darknet53.conv.74 -i 0 | tee train_log.txt

In [ ]:
!cat ../../input/covid19jpg/CovidJPG/train/8951eac18958.txt

### 4.2. My Plot of Training Loss
It's a loss graph up to about 2000 iteration. Since it tooks too long on kaggle kernel, I brought it. When learning, don't be surprised of big loss values at the beginning. Stay calm and It'll go down. Please See the following loss graph.

In [ ]:
!wget --no-check-certificate -q "https://docs.google.com/uc?export=download&id=1OhnlV3s7r6xsEme6DKkNYjcYjsl-C_Av" -O train_log.txt

In [ ]:
iters = []
losses = []
total_losses = []
with open("train_log.txt", 'r') as f:
    for i,line in enumerate(f):
        if "images" in line:
            iters.append(int(line.strip().split()[0].split(":")[0]))
            losses.append(float(line.strip().split()[2]))        
            total_losses.append(float(line.strip().split()[1].split(',')[0]))

plt.figure(figsize=(20, 5))
plt.subplot(1,2,1)
sns.lineplot(iters, total_losses, label="totla loss")
sns.lineplot(iters, losses, label="avg loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")

plt.subplot(1,2,2)
sns.lineplot(iters, total_losses, label="totla loss")
sns.lineplot(iters, losses, label="avg loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.ylim([0, 4.05])

## 5. How to use trainined YOLOv3 for test images (command line)

### 5.0. Copy sample test image

In [ ]:
ex_patient_id = annots[annots.Target == 1].patientId.values[2]
shutil.copy(ex_img_path, "test.jpg")
print(ex_patient_id)

### 5.1. Load trained model (at 15300 iteration)
Since i uploaded the weights file (large big file) on my google drive, the command is very very long ...
* It's a weight file at 15300 iteration, which I made submission file with. If you use this weight, you'll get a score of 0.141LB.
  * Up to 15300 iteration, It takes about 8 hours.
    * In .cfg file, I set 'batch' and 'subdivisions' as 64 and 8 respectively.
    * Up to 1000 iteration from 0, it takes about 1h with **one** Tesla P100 GPU.      **(1000 iter/h)**
    * Up to 15300 iteration from 1000, it takes about 7h with **four** Tesla P100 GPU. **(2043 iter/h)**

In [ ]:
!wget --load-cookies /tmp/cookies.txt -q "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1FDzMN-kGVYCvBeDKwemAazldSVkAEFyd' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1FDzMN-kGVYCvBeDKwemAazldSVkAEFyd" -O backup/rsna_yolov3_15300.weights && rm -rf /tmp/cookies.txt

In [ ]:
!ls -alsth backup

### 5.2. cfg file for test (not for training)

In [ ]:
!wget --no-check-certificate -q "https://docs.google.com/uc?export=download&id=10Yk6ZMAKGz5LeBbikciALy82aK3lX-57" -O cfg/rsna_yolov3.cfg_test

In [ ]:
!cd darknet && ./darknet detector test ../cfg/rsna.data ../cfg/rsna_yolov3.cfg_test ../backup/rsna_yolov3_15300.weights ../test.jpg -thresh 0.005

In [ ]:
# ![](predictions.jpg)
plt.imshow(cv2.imread("./darknet/predictions.jpg"))

## 6. Generate Submission Files with YOLOv3 Python Wrapper

### 6.0. Download darknet python wrapper (darknet.py)
* Basically, you can use darknet/python/darknet.py files. However it'll show error.
* So, I edited the darknet.py. There are two main modifications.
  * Change print statement to print function for python3
  * Edit dynamic library('libdarknet.so') file path
* I leaved '# ===' marks where i edited in darknet.py. For example,
```
# ==============================================================================
#lib = CDLL("/home/pjreddie/documents/darknet/libdarknet.so", RTLD_GLOBAL)
darknet_lib_path = os.path.join(os.getcwd(), "darknet", "libdarknet.so")
lib = CDLL(darknet_lib_path, RTLD_GLOBAL)
# ==============================================================================
```

In [ ]:
!wget --no-check-certificate -q "https://docs.google.com/uc?export=download&id=1-KTV7K9G1bl3SmnLnzmpkDyNt6tDmH7j" -O darknet.py

### 6.1. Load darknet python wrapper module

In [ ]:
from darknet import *

### 6.2. Generate submission files
* When making submission files, be aware of label format which is different in yolo.

In [ ]:
threshold = 0.2

In [ ]:
submit_file_path = "submission.csv"
cfg_path = os.path.join(cfg_dir, "rsna_yolov3.cfg_test")
weight_path = os.path.join(backup_dir, "rsna_yolov3_15300.weights")

test_img_list_path = os.path.join(metadata_dir, "te_list.txt")

In [ ]:
gpu_index = 0
net = load_net(cfg_path.encode(),
               weight_path.encode(), 
               gpu_index)
meta = load_meta(data_extention_file_path.encode())

In [ ]:
submit_dict = {"patientId": [], "PredictionString": []}

with open(test_img_list_path, "r") as test_img_list_f:
    # tqdm run up to 1000(The # of test set)
    for line in tqdm(test_img_list_f):
        patient_id = line.strip().split('/')[-1].strip().split('.')[0]

        infer_result = detect(net, meta, line.strip().encode(), thresh=threshold)

        submit_line = ""
        for e in infer_result:
            confi = e[1]
            w = e[2][2]
            h = e[2][3]
            x = e[2][0]-w/2
            y = e[2][1]-h/2
            submit_line += "{} {} {} {} {} ".format(confi, x, y, w, h)

        submit_dict["patientId"].append(patient_id)
        submit_dict["PredictionString"].append(submit_line)

pd.DataFrame(submit_dict).to_csv(submit_file_path, index=False)

In [ ]:
# !ls -lsht
!rm -rf darknet images labels metadata backup cfg
!rm -rf train_log.txt darknet53.conv.74 darknet.py darknet_gpu
!rm -rf test.jpg
!rm -rf __pycache__ .ipynb_checkpoints

In [ ]:
!ls -alsht

## 7. Future works & Etc

### Future works (Things to try)
* Image augmentation
* More training
* Utilizing the not labeled images because we got rid of not labeled images above

### ETC
* For a private matter, i can not proceed RSNA task after 09/27. If you have any ideas, questions and problems with this kernel after 09/27, Please leave those things anyway~! Collaborator '@John Byun' will reply to your comments.